In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#List of all teams and the components of the URL
url =['https://www.cbssports.com/nhl/teams/','/roster/']
teams = [
    ['BOS', 'boston-bruins'],
    ['BUF', 'buffalo-sabres'],
    ['DET', 'detroit-red-wings'],
    ['FLA', 'florida-panthers'],
    ['MON', 'montreal-canadiens'],
    ['OTT', 'ottawa-senators'],
    ['TB', 'tampa-bay-lightning'],
    ['TOR', 'toronto-maple-leafs'],
    ['ARI', 'arizona-coyotes'],
    ['CHI', 'chicago-blackhawks'],
    ['COL', 'colorado-avalanche'],
    ['DAL', 'dallas-stars'],
    ['MIN', 'minnesota-wild'],
    ['NSH', 'nashville-predators'],
    ['STL', 'st-louis-blues'],
    ['WPG', 'winnipeg-jets'],
    ['CAR', 'carolina-hurricanes'],
    ['CLB', 'columbus-blue-jackets'],
    ['NYI', 'new-york-islanders'],
    ['NYR', 'new-york-rangers'],
    ['NJ', 'new-jersey-devils'],
    ['PHI', 'philadelphia-flyers'],
    ['PIT', 'pittsburgh-penguins'],
    ['WAS', 'washington-capitals'],
    ['ANA', 'anaheim-ducks'],
    ['CGY', 'calgary-flames'],
    ['EDM', 'edmonton-oilers'],
    ['LA', 'los-angeles-kings'],
    ['SJ', 'san-jose-sharks'],
    ['SEA', 'seattle-kraken'],
    ['VAN', 'vancouver-canucks'],
    ['LV', 'vegas-golden-knights']]

In [3]:
#Makes an array of the team roster urls
pages = []
for team in teams:
    pages.append(f'{url[0]}{team[0]}/{team[1]}{url[1]}')

In [4]:
#takes beautiful soup item and team id to create an array of players for a whole team and returns the array
def getPlayersStats(players, teamID):
    playerStats = []
    for player in players:
        number = player.find_all('td', class_ = 'TableBase-bodyTd')[0].text.strip()
        name = ((player.find('span', class_ = 'CellPlayerName--long')).find('a').text).split(' ')
        fName = name[0]
        lName  = name[1]
        pos = player.find_all('td', class_ = 'TableBase-bodyTd')[2].text.strip()
        shot = player.find_all('td', class_ = 'TableBase-bodyTd')[3].text.strip()
        ht = player.find_all('td', class_ = 'TableBase-bodyTd')[4].text.strip()
        wt = player.find_all('td', class_ = 'TableBase-bodyTd')[5].text.strip()
        dob = player.find_all('td', class_ = 'TableBase-bodyTd')[6].text.strip()[:-5]
        birthplace = player.find_all('td', class_ = 'TableBase-bodyTd')[7].text.strip()
        playerID = f'{fName[:1]}{lName[:4]}{number}'
        team = teamID
        playerStats.append([playerID, fName, lName, number, pos, shot, ht, wt, dob, birthplace, team])
    return playerStats

In [5]:
#takes the list of pages itterates through and passes them to the get player stats function
#returns a list complete list of player stats
def get_teamPages(pages):
    count = 0
    playerList = []
    for page in pages:
        team = teams[count][0]
        url = requests.get(page)
        soup = BeautifulSoup(url.content, "html.parser")
        players = soup.find_all('tr', class_='TableBase-bodyTr')
        playerList = playerList + getPlayersStats(players,team)
        count+=1
    return playerList


In [6]:
df = pd.DataFrame(get_teamPages(pages))
df = df.rename(columns={0:'playerID', 1:'firstName', 2:'lastName', 3:'number', 4:'position', 5:'shot', 6:'height', 7:'weight', 8:'dob', 9: 'birthplace', 10:'teamID'})
df = df.set_index('playerID')

In [11]:
#locates all players on the team with ID WPG - Winapeg Jets
df.loc[df['teamID'].str.contains('LA')]

,firstName,lastName,number,position,shot,height,weight,dob,birthplace,teamID
playerID,,,,,,,,,,
ABark16,Aleksander,Barkov,16,C,R,6-3,215,9/2/1995,"Tampere, Finland",FLA
SBenn9,Sam,Bennett,9,C,R,6-1,195,6/20/1996,"Holland Landing, ON",FLA
NCous21,Nick,Cousins,21,C,R,5-11,183,7/20/1993,"Belleville, ON",FLA
SLore18,Steven,Lorentz,18,C,R,6-4,206,4/13/1996,"Kitchener, ON",FLA
ALund15,Anton,Lundell,15,C,R,6-1,185,10/3/2001,"Espoo, Finland",FLA
ELuos27,Eetu,Luostarinen,27,C,R,6-3,190,9/2/1998,"Siilinjarvi, Finland",FLA
SRein13,Sam,Reinhart,13,C,R,6-2,193,11/6/1995,"West Vancouver, BC",FLA
ERodr17,Evan,Rodrigues,17,C,R,5-11,184,7/28/1993,"Toronto, ON",FLA
KSten28,Kevin,Stenlund,28,C,R,6-4,215,9/20/1996,"Stockholm, Sweden",FLA


In [8]:
filePath = 'players.csv'

df.to_csv(filePath)

"filePath = 'players.csv'\n\ndf.to_csv(filePath)"

In [12]:
df.describe()

,firstName,lastName,number,position,shot,height,weight,dob,birthplace,teamID
count,853,853,853,853,853,853,853,853,853,853
unique,415,787,98,5,2,15,82,785,527,32
top,Ryan,Smith,—,D,R,6-2,195,1/13/1997,"Toronto, ON",OTT
freq,18,5,40,277,773,153,37,3,26,33
